## Data Wrangling with Python: Intro to Pandas
Note: Notebook adapted from [here](https://github.com/EricElmoznino/lighthouse_pandas_tutorial/blob/master/pandas_tutorial.ipynb) & [here](https://github.com/sedv8808/LighthouseLabs/tree/main/W02D2) & from LHL's [21 Day Data Challenge](https://data-challenge.lighthouselabs.ca/start)
#### Instructor: Simon Dawkins
#### Date: January 18, 2022

**Agenda:**
 - Why Pandas?
 - Pandas Basics
     - Pandas Series vs. Pandas DataFrames
     - .loc() vs. iloc()
 - Pandas Advance
     - Filtering
     - Group-bys
 - Pandas Exercises
     - Challenge 1
     - Challenge 2

In [46]:
import pandas as pd
pokemon = pd.read_csv('pokemon.csv')
df = pd.read_csv('winequality-red_2.csv')
df = pd.read_csv('video_games.csv')

### Pandas: Why Pandas? What is it? 


To do data anlaysis with Python, Pandas is a great tool to for dealing with data in a tabular and time series formats. Designed by Wes McKinney as an attempt to port R's dataframes to python. 

- Python Package for working with **tables**
- Similar to SQL & Excel
    - Faster sometimes - more optimized for non-distributed platforms
    - More features to manipulate, transform, and aggregate data
- Easy to handle messy and missing data
- Great at working with large datasets
- When combing with other Python libraries, it's fairly easy to create beautiful and customazied visuals. Easy integration with Matplotlib, Seaborn, Plotly.
- Easy integration with machine learning plugins (sckit-learn, etc)
    
    
-----------
To read more about, Wes McKinney, the creator of Pandas, check out the article below.

1. https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/

--------------


## Think of how we would try to represent a table in Python?


In [1]:
#A dicitonary of lists example
students = {
    'student_id': [1, 2, 3, 4,5,6],
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'],
    'course_mark': [82, 100, 12, 76, 46, 20],
    'species': ['cat', 'human', 'cat', 'human', 'human', 'human']
}

**What are some operations we might want to do on this data?**

- 1. Select a subset of columns
- 2. Filter out some rows based on an attribute
- 3. Group by some attribute
- 4. Compute some aggregate values within groups
- 5. Save to a file

How about we try out one of these to see how easy it is

### Return a table with the mean course mark per-species with vanilla Python

In [2]:
# Return a table with the mean course mark per species

species_sums = {} # Tables of Sums
species_counts = {} # Count per Species
for i in range(len(students['species'])):  # iterating over the rows
    species = students['species'][i] # every row number I get species 
    course_mark = students['course_mark'][i] #  and course mark
    if species not in species_sums: # Intializing Species if not in list
        species_sums[species] = 0
        species_counts[species] = 0
    species_sums[species] += course_mark # Add each course mark for each species
    species_counts[species] += 1 

species_means = {}
                                  
for species in species_sums: # for every unique species we found
    species_means[species] = species_sums[species] / species_counts[species] #sum/count

species_means

{'cat': 47.0, 'human': 60.5}

- Did you like looking at this? 
- Does this look fun to do? 
- what about with 10000 rows and 50 columns?
- Super Tiring.

## Pandas Version

In [3]:
import pandas as pd

# Can take in a dictionry of list to instantiate a DataFrame
students = pd.DataFrame(students)
students

,student_id,name,course_mark,species
0,1,Daenerys,82,cat
1,2,Jon,100,human
2,3,Arya,12,cat
3,4,Sansa,76,human
4,5,Eddard,46,human
5,6,Khal Drogo,20,human


In [4]:
species_means = students[['species', 'course_mark']].groupby('species').mean()
# species_means = students.groupby('species')['course_mark'].mean()
species_means

,course_mark
species,
cat,47.0
human,60.5


### Dissecting the above code:


In [5]:
#Step 1: Filter out the columns we want to keep
students_filtered = students[['species','course_mark']]
students_filtered

,species,course_mark
0,cat,82
1,human,100
2,cat,12
3,human,76
4,human,46
5,human,20


In [6]:
# Step 2: Group by species column
students_grouped_by_species = students_filtered.groupby('species') 
students_grouped_by_species

In [7]:
#Step 3: Specify how to aggregate the course-mark column
species_means = students_grouped_by_species.mean()

In [8]:
species_means

,course_mark
species,
cat,47.0
human,60.5


#### As shown, Pandas makes use of vectorized operations. 


- Rather than use for-loops, we specify the operation that will apply to the structure as a whole (i.e. all the rows)
- By vectorizing, **the code becomes more concise and more readable**
- Pandas is optimized for vectorized operations (parallel vs. serial computation), which makes them **much faster**
- It is almost always possible to vectorize operations on Pandas data types - series have a single data type


### Getting Started: Pandas Series & Pandas DataFrames

There are two Pandas data types of interest:

- Series (column)
    - A pandas series is similar to an array but it has an index. The index is constant, and doesnt change through the operations we apply to the series. 
- DataFrame (table)
    - A pandas dataframe is an object that is similar to a collection of pandas series.

In [9]:
# One way to construct a Series
grades = pd.Series([82, 100, 12, 76, 46, 20]) 
grades

0     82
1    100
2     12
3     76
4     46
5     20
dtype: int64

In [10]:
#We can specify some index when building a series. 
grades = pd.Series([82, 100, 12, 76, 46, 20], 
                   index = ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'] ) 

grades

Daenerys       82
Jon           100
Arya           12
Sansa          76
Eddard         46
Khal Drogo     20
dtype: int64

In [11]:
print("The values:", grades.values)
print("The indexes:", grades.index)

The values: [ 82 100  12  76  46  20]
The indexes: Index(['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'], dtype='object')


**Note:** The underlying index is still 0, 1, 2, 3.... and we can still index on that:

In [12]:
grades[2]

12

### Pandas DataFrames

In [47]:
# One way to construct a DataFrame
df = pd.DataFrame({
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']},
    index=[1412, 94, 9351, 14])
df

94

#### Reading a CSV file

We'll use the function `read_csv()` to load the data into our notebook

- The `read_csv()` function can read data from a locally saved file or from a URL
- We'll store the data as a variable `df_pokemon`

In [14]:
pokemon = pd.read_csv('pokemon.csv')

In [15]:
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**What do we see here?**
- Each row of the table is an observation, containing data of a single pokemon

In [48]:
# shape
pokemon.shape

(800, 13)

For large DataFrames, it's often useful to display just the first few or last few rows:

In [49]:
# head
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [50]:
# tail
pokemon.tail(10)                           

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False
791,715,Noivern,Flying,Dragon,535,85,70,80,97,80,123,6,False
792,716,Xerneas,Fairy,NaN,680,126,131,95,131,98,99,6,True
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


## Data at a Glance

`pandas` provides many ways to quickly and easily summarize your data:
- How many rows and columns are there?
- What are all the column names and what type of data is in each column?
- How many values are missing in each column or row?
- Numerical data: What is the average and range of the values?
- Text data: What are the unique values and how often does each occur?

### Peeking into the pokemon dataset

- Similar with getting familar with SQL tables, it is often a great idea to look at the pandas dataframes we are working with. Below are some of the basic methods to glance at a dataset. 

In [52]:
# Getting the Columns, attribute
pokemon.columns
type(pokemon.columns)

pandas.core.indexes.base.Index

In [53]:
# Getting Summary Statistics, method
# will only output for numerical columns
# not for ordinal or categorical values
pokemon.describe()


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [58]:
pokemon[['Total', 'HP']].describe()

,Total,HP
count,800.00000,800.000000
mean,435.10250,69.258750
std,119.96304,25.534669
min,180.00000,1.000000
25%,330.00000,50.000000
50%,450.00000,65.000000
75%,515.00000,80.000000
max,780.00000,255.000000


In [55]:
# accessing columns with dot
# can only access once column at a time
# keywords will also interfere with this call
pokemon.Total.describe()

count    800.00000
mean     435.10250
std      119.96304
min      180.00000
25%      330.00000
50%      450.00000
75%      515.00000
max      780.00000
Name: Total, dtype: float64

In [61]:
# Checking for Missing Data, method
pokemon.isnull().sum()
pokemon.isnull().sum() / len(pokemon) # percentage of null values for each column

#             0.0000
Name          0.0000
Type 1        0.0000
Type 2        0.4825
Total         0.0000
HP            0.0000
Attack        0.0000
Defense       0.0000
Sp. Atk       0.0000
Sp. Def       0.0000
Speed         0.0000
Generation    0.0000
Legendary     0.0000
dtype: float64

In [62]:
# getting info
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## The .loc() vs .iloc() method


To select rows and columns at the same time, we use the syntax `.loc[ROWS SLICE, COLUMNS SLICE]`:

In [63]:
# simple slice
# Notice the square brackets on loc, this is a method
# despite having square brackets
# The brackets mimic the way python calls on index or key values
pokemon.loc[20:30, ['Attack']]

,Attack
20,45
21,60
22,80
23,80
24,56
25,81
26,60
27,90
28,60
29,85


In [65]:
# Getting more than one column
# The range given in the 1 axis (row) will always
# default to the implicit index and not the
# integers passed to name the rows.
pokemon.loc[20:30, ['Name', 'Legendary']]

,Name,Legendary
20,Pidgey,False
21,Pidgeotto,False
22,Pidgeot,False
23,PidgeotMega Pidgeot,False
24,Rattata,False
25,Raticate,False
26,Spearow,False
27,Fearow,False
28,Ekans,False
29,Arbok,False


In [66]:
# pass a list for the rows
pokemon.loc[[2,3,4,5,3,80],['Name', 'Legendary']]

,Name,Legendary
2,Venusaur,False
3,VenusaurMega Venusaur,False
4,Charmander,False
5,Charmeleon,False
3,VenusaurMega Venusaur,False
80,Geodude,False


In [78]:
# pass a range range of column values
pokemon.loc[[1,3,2],'Name':'Attack':1]

,Name,Type 1,Type 2,Total,HP,Attack
1,Ivysaur,Grass,Poison,405,60,62
3,VenusaurMega Venusaur,Grass,Poison,625,80,100
2,Venusaur,Grass,Poison,525,80,82


In [82]:
# iloc is used for integer based indexing
# insteaad of loc's special indexing, iloc using vanilla indexing
# inclusive vs exclusive slicing
pokemon.iloc[0:50, 0:4]

,#,Name,Type 1,Type 2
0,1,Bulbasaur,Grass,Poison
1,2,Ivysaur,Grass,Poison
2,3,Venusaur,Grass,Poison
3,3,VenusaurMega Venusaur,Grass,Poison
4,4,Charmander,Fire,NaN
5,5,Charmeleon,Fire,NaN
6,6,Charizard,Fire,Flying
7,6,CharizardMega Charizard X,Fire,Dragon
8,6,CharizardMega Charizard Y,Fire,Flying
9,7,Squirtle,Water,NaN


### Modifying a Column or Creating a new column

In [83]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [86]:
# Make sure make a copy of your dataframe before doing any modification, this is a method
# This is generaly best practice, and it will be easier for you to keep track of the various
# changes and states of your dataframe. 
# do not do this:
pokemon2 = pokemon # this will only reference the reference of the original dataset; it does not create another dataset 

# do this instead:
pokemon2 = pokemon.copy()

In [87]:
# create a new column
# Combine Attack + Special Attack
pokemon2['Total Attack'] = pokemon2['Attack'] + pokemon2['Sp. Atk']

In [91]:
pokemon2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,1272,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,1620,60,62,63,80,80,60,1,False,142
2,3,Venusaur,Grass,Poison,2100,80,82,83,100,100,80,1,False,182
3,3,VenusaurMega Venusaur,Grass,Poison,2500,80,100,123,122,120,80,1,False,222
4,4,Charmander,Fire,NaN,1236,39,52,43,60,50,65,1,False,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,2400,50,100,150,100,150,50,6,True,200
796,719,DiancieMega Diancie,Rock,Fairy,2800,50,160,110,160,110,110,6,True,320
797,720,HoopaHoopa Confined,Psychic,Ghost,2400,80,110,60,150,130,70,6,True,260
798,720,HoopaHoopa Unbound,Psychic,Dark,2720,80,160,60,170,130,80,6,True,330


In [89]:
# modify existing column
pokemon2['Total'] = pokemon2['Total'] * 2

In [90]:
pokemon2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,1272,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,1620,60,62,63,80,80,60,1,False,142
2,3,Venusaur,Grass,Poison,2100,80,82,83,100,100,80,1,False,182
3,3,VenusaurMega Venusaur,Grass,Poison,2500,80,100,123,122,120,80,1,False,222
4,4,Charmander,Fire,NaN,1236,39,52,43,60,50,65,1,False,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,2400,50,100,150,100,150,50,6,True,200
796,719,DiancieMega Diancie,Rock,Fairy,2800,50,160,110,160,110,110,6,True,320
797,720,HoopaHoopa Confined,Psychic,Ghost,2400,80,110,60,150,130,70,6,True,260
798,720,HoopaHoopa Unbound,Psychic,Dark,2720,80,160,60,170,130,80,6,True,330


In [92]:
# Modify Data Frame with .loc() method
pokemon2.loc[[1,2,4,5], ['Name']] = 'Simon'

In [98]:
pokemon2.loc[1:5,'Name': 'Legendary']

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Simon,Grass,Poison,1620,60,62,63,80,80,60,1,False
2,Simon,Grass,Poison,2100,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,2500,80,100,123,122,120,80,1,False
4,Simon,Fire,NaN,1236,39,52,43,60,50,65,1,False
5,Simon,Fire,NaN,1620,58,64,58,80,65,80,1,False


### Sort_values() & value_counts()

1. ***df.sort_values()***
2. ***df.value_counts()***


The ***pandas.sort_values()*** allows us to reorder our dataframe in an ascending or descending order given a column for pandas to work from. This is similar to the excel sort function.

```python
import pandas as pd
df = pd.read_csv('random.csv')
df


df.sort_values(by=['some_column'], ascending = True)
```
In the above code snippet, we are sorting our *random.csv* pandas data frame by the column *some_column* in ascending order. To read more on the ***df.sort_values()*** function, read this [article](https://datatofish.com/sort-pandas-dataframe/).

The second function is ***df.value_counts()***, it allows us to count how many times a specific value/item occurred in the dataframe. This function is best used on a specific column on a data frame, ideally on a column representing categorical data. Categorical data refers to a statistical data type consisting of categorical variables. 

```python
df['column'].value_counts()
```

To read more on some of the advanced functionalities of ***df.value_counts()***, please refer to the pandas documentation or this [article](https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6).

In [99]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [103]:
# list Unique Values
pokemon.sort_values(by='Generation', ascending=False)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True
738,670,Floette,Fairy,NaN,371,54,45,47,75,98,52,6,False
740,672,Skiddo,Grass,NaN,350,66,65,48,62,57,52,6,False
741,673,Gogoat,Grass,NaN,531,123,100,62,97,81,68,6,False
742,674,Pancham,Fighting,NaN,348,67,82,62,46,48,43,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,102,Exeggcute,Grass,Psychic,325,60,40,80,60,45,40,1,False
109,101,Electrode,Electric,NaN,480,60,50,70,80,80,140,1,False
108,100,Voltorb,Electric,NaN,330,40,30,50,55,55,100,1,False
107,99,Kingler,Water,NaN,475,55,130,115,50,50,75,1,False


In [107]:
# How many unique Values
pokemon.sort_values(by=['HP', 'Attack'], ascending=[False, True])

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,1,False
217,202,Wobbuffet,Psychic,NaN,405,190,33,58,33,58,33,2,False
351,321,Wailord,Water,NaN,500,170,90,45,90,45,60,3,False
655,594,Alomomola,Water,NaN,470,165,75,80,40,45,65,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,439,Mime Jr.,Psychic,Fairy,310,20,25,45,70,90,60,4,False
186,172,Pichu,Electric,NaN,205,20,40,15,35,35,60,2,False
388,355,Duskull,Ghost,NaN,295,20,40,90,30,90,25,3,False
55,50,Diglett,Ground,NaN,265,10,55,25,35,45,95,1,False


In [111]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [112]:
type(pokemon['Type 1'].value_counts()) # the value_count method returns a series so any methods that can be applied there can be applied here.

pandas.core.series.Series

In [113]:
pokemon['Type 1'].value_counts().sort_index()

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
Name: Type 1, dtype: int64

In [116]:
pokemon['Type 1'].unique() # acts like a set

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [115]:
pokemon['Type 1'].nunique() # 'n'unique will return the count

18

### How to Query or Filter Data with Conditions?

- We can extract specific data from our dataframe based on a specific condition. We will be using the syntax below. Pandas will return a subset of the dataframe based on the given condition. 

```python
df[<insert_condition>]
```

Conditions follow the generic boolean logic in Python. Below is a cheat sheet python boolean logic.

**Conditional Logic:** 

Conditional logic refers to the execution of different actions based on whether a certain condition is met. In programming, these conditions are expressed by a set of symbols called **Boolean Operators**. 

| Boolean Comparator | Example | Meaning                         |
|--------------------|---------|---------------------------------|
| >                  | x > y   | x is greater than y             |
| >=                 | x >= y  | x is greater than or equal to y |
| <                  | x < y   | x is less than y                |
| <=                 | x <= y  | x is less than or equal to y    |
| !=                 | x != y  | x is not equal to y             |
| ==                 | x == y  | x is equal to y                 |




In [125]:
# Step 1: Create a filter
filter1 = (pokemon['Total'] >= 500) # boolean operations must be enclosed in parenthesis

In [126]:
# Step 2: Apply Filter
pokemon[filter1] # like a WHERE method in SQL

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Alternativly we can use the pandas **.where()** function, which has the following syntax.


```python 
df.where(<condition>, <What to fill inplace where the condition is not True>) # default is Nan
```

In [129]:
pokemon.where(filter1, 'Strong Pokemon') # this is when you want to do something to instances

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
1,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


### Grouping and Aggregation 

Grouping and aggregation can be used to calculate statistics on groups in the data.

**Common Aggregation Functions**
- mean()
- median()
- sum()
- count()


In [131]:
# simple groupby
pokemon.groupby('Type 1').mean()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
Type 1,,,,,,,,,,
Bug,334.492754,378.927536,56.884058,70.971014,70.724638,53.869565,64.797101,61.681159,3.217391,0.000000
Dark,461.354839,445.741935,66.806452,88.387097,70.225806,74.645161,69.516129,76.161290,4.032258,0.064516
Dragon,474.375000,550.531250,83.312500,112.125000,86.375000,96.843750,88.843750,83.031250,3.875000,0.375000
Electric,363.500000,443.409091,59.795455,69.090909,66.295455,90.022727,73.704545,84.500000,3.272727,0.090909
Fairy,449.529412,413.176471,74.117647,61.529412,65.705882,78.529412,84.705882,48.588235,4.117647,0.058824
Fighting,363.851852,416.444444,69.851852,96.777778,65.925926,53.111111,64.703704,66.074074,3.370370,0.000000
Fire,327.403846,458.076923,69.903846,84.769231,67.769231,88.980769,72.211538,74.442308,3.211538,0.096154
Flying,677.750000,485.000000,70.750000,78.750000,66.250000,94.250000,72.500000,102.500000,5.500000,0.500000
Ghost,486.500000,439.562500,64.437500,73.781250,81.187500,79.343750,76.468750,64.343750,4.187500,0.062500


- By default, `groupby()` assigns the variable that we're grouping on (in this case `Type 1`) to the index of the output data
- If we use the keyword argument `as_index=False`, the grouping variable is instead assigned to a regular column
  - This can be useful in some situations, such as data visualization functions which expect the relevant variables to be in columns rather than the index

In [132]:
# grouby with as_index set to False
pokemon.groupby('Type 1', as_index=False).sum()

,Type 1,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bug,23080,26146,3925,4897,4880,3717,4471,4256,222,0
1,Dark,14302,13818,2071,2740,2177,2314,2155,2361,125,2
2,Dragon,15180,17617,2666,3588,2764,3099,2843,2657,124,12
3,Electric,15994,19510,2631,3040,2917,3961,3243,3718,144,4
4,Fairy,7642,7024,1260,1046,1117,1335,1440,826,70,1
5,Fighting,9824,11244,1886,2613,1780,1434,1747,1784,91,0
6,Fire,17025,23820,3635,4408,3524,4627,3755,3871,167,5
7,Flying,2711,1940,283,315,265,377,290,410,22,2
8,Ghost,15568,14066,2062,2361,2598,2539,2447,2059,134,2
9,Grass,24141,29480,4709,5125,4956,5425,4930,4335,235,3


In [140]:
pokemon.groupby('Type 1', as_index=False)['Attack'].sum()

,Type 1,Attack
0,Bug,4897
1,Dark,2740
2,Dragon,3588
3,Electric,3040
4,Fairy,1046
5,Fighting,2613
6,Fire,4408
7,Flying,315
8,Ghost,2361
9,Grass,5125


We can use the `agg` method to compute multiple aggregate functions on our data, for example minimum, maximum and mean attack in Type 1

In [141]:
pokemon.groupby(['Type 1', 'Legendary']).mean() # creates multi-indexing table

# notice below that bug type doesn't have a Legendary type and so on...

#       Total          HP      Attack  \
Type 1   Legendary                                                   
Bug      False      334.492754  378.927536   56.884058   70.971014   
Dark     False      451.517241  432.344828   64.655172   86.862069   
         True       604.000000  640.000000   98.000000  110.500000   
Dragon   False      447.350000  476.850000   72.650000  103.400000   
         True       519.416667  673.333333  101.083333  126.666667   
Electric False      358.050000  429.750000   57.325000   66.125000   
         True       418.000000  580.000000   84.500000   98.750000   
Fairy    False      432.875000  396.500000   70.875000   57.187500   
         True       716.000000  680.000000  126.000000  131.000000   
Fighting False      363.851852  416.444444   69.851852   96.777778   
Fire     False      322.957447  442.127660   67.085106   82.191489   
         True       369.200000  608.000000   96.400000  109.000000   
Flying   False      714.500000  390.000000   62.500000   50.000000   
         True       641.000000  580.000000   79.000000  107.500000   
Ghost    False      486.466667  423.533333   58.733333   71.366667   
         True       487.000000  680.000000  150.000000  110.000000   
Grass    False      336.074627  413.432836   65.940299   72.119403   
         True       541.333333  593.333333   97.000000   97.666667   
Ground   False      333.750000  405.714286   70.821429   88.000000   
         True       514.000000  660.000000   94.500000  150.000000   
Ice      False      438.318182  420.136364   70.818182   73.227273   
         True       261.000000  580.000000   85.000000   67.500000   
Normal   False      315.625000  395.572917   76.489583   72.083333   
         True       489.500000  695.000000  115.000000  140.000000   
Poison   False      251.785714  399.142857   67.250000   74.678571   
Psychic  False      374.093023  423.000000   67.720930   54.953488   
         True       401.428571  638.571429   79.571429  122.142857   
Rock     False      370.650000  437.625000   65.125000   89.925000   
         True       613.500000  615.000000   67.750000  122.250000   
Steel    False      437.913043  466.434783   60.434783   92.086957   
         True       471.250000  610.000000   92.750000   96.250000   
Water    False      300.490741  421.398148   71.120370   72.777778   
         True       373.250000  675.000000   97.500000  111.250000   

                       Defense     Sp. Atk     Sp. Def       Speed  Generation  
Type 1   Legendary                                                              
Bug      False       70.724638   53.869565   64.797101   61.681159    3.217391  
Dark     False       68.689655   70.620690   67.827586   73.689655    3.965517  
         True        92.500000  133.000000   94.000000  112.000000    5.000000  
Dragon   False       78.150000   72.900000   77.400000   72.350000    3.750000  
         True       100.083333  136.750000  107.916667  100.833333    4.083333  
Electric False       65.425000   86.275000   72.325000   82.275000    3.275000  
         True        75.000000  127.500000   87.500000  106.750000    3.250000  
Fairy    False       63.875000   75.250000   83.875000   45.437500    4.000000  
         True        95.000000  131.000000   98.000000   99.000000    6.000000  
Fighting False       65.925926   53.111111   64.703704   66.074074    3.370370  
Fire     False       64.531915   86.000000   69.042553   73.276596    3.234043  
         True        98.200000  117.000000  102.000000   85.400000    3.000000  
Flying   False       57.500000   71.000000   60.000000   89.000000    6.000000  
         True        75.000000  117.500000   85.000000  116.000000    5.000000  
Ghost    False       79.266667   77.300000   74.233333   62.633333    4.200000  
         True       110.000000  110.000000  110.000000   90.000000    4.000000  
Grass    False       70.283582   76.343284   69.044776   59.701493    3.313433  
         True        82.333333  103.333

In [142]:
# combine multiple agg functions
pokemon.groupby('Type 1')['Attack'].agg(['mean', 'max', 'min'])

,mean,max,min
Type 1,,,
Bug,70.971014,185,10
Dark,88.387097,150,50
Dragon,112.125000,180,50
Electric,69.090909,123,30
Fairy,61.529412,131,20
Fighting,96.777778,145,35
Fire,84.769231,160,30
Flying,78.750000,115,30
Ghost,73.781250,165,30


We can also use `agg` to compute different agg functions for different columns:

In [146]:
agg_dict = {
    'Attack' : 'mean',
    'Defense': ['min', 'max']
}

In [147]:
pokemon.groupby('Type 1').agg(agg_dict) # turns the column 'Type1' into the row index

Attack Defense     
                mean     min  max
Type 1                           
Bug        70.971014      30  230
Dark       88.387097      30  125
Dragon    112.125000      35  130
Electric   69.090909      15  115
Fairy      61.529412      28   95
Fighting   96.777778      30   95
Fire       84.769231      37  140
Flying     78.750000      35   80
Ghost      73.781250      30  145
Grass      73.214286      30  131
Ground     95.750000      25  160
Ice        72.750000      15  184
Normal     73.469388       5  126
Poison     74.678571      35  120
Psychic    71.456140      15  160
Rock       92.863636      40  200
Steel      92.703704      50  230
Water      74.151786      20  180

In [148]:
pokemon.groupby(['Type 1', 'Legendary'], as_index = False).agg(agg_dict)

Type 1 Legendary      Attack Defense     
                              mean     min  max
0        Bug     False   70.971014      30  230
1       Dark     False   86.862069      30  125
2       Dark      True  110.500000      90   95
3     Dragon     False  103.400000      35  130
4     Dragon      True  126.666667      80  121
5   Electric     False   66.125000      15  115
6   Electric      True   98.750000      70   85
7      Fairy     False   57.187500      28   95
8      Fairy      True  131.000000      95   95
9   Fighting     False   96.777778      30   95
10      Fire     False   82.191489      37  140
11      Fire      True  109.000000      85  120
12    Flying     False   50.000000      35   80
13    Flying      True  107.500000      70   80
14     Ghost     False   71.366667      30  145
15     Ghost      True  110.000000     100  120
16     Grass     False   72.119403      30  131
17     Grass      True   97.666667      72  100
18    Ground     False   88.000000      25  130
19    Ground      True  150.000000      90  160
20       Ice     False   73.227273      15  184
21       Ice      True   67.500000     100  100
22    Normal     False   72.083333       5  126
23    Normal      True  140.000000     110  120
24    Poison     False   74.678571      35  120
25   Psychic     False   54.953488      15  120
26   Psychic      True  122.142857      20  160
27      Rock     False   89.925000      40  168
28      Rock      True  122.250000      90  200
29     Steel     False   92.086957      50  230
30     Steel      True   96.250000     100  150
31     Water     False   72.777778      20  180
32     Water      True  111.250000      90  115

### Challenge 1 (20 minutes)

Let's play around with Pandas on a more intricate dataset: a dataset on wines!

**Challenge 14 from the 21 Day Data Challenge** 

Dot's neighbour said that he only likes wine from Stellenbosch, Bordeaux, and the Okanagan Valley, and that the sulfates can't be that high. The problem is, Dot can't really afford to spend tons of money on the wine. Dot's conditions for searching for wine are: 
1. Sulfates cannot be higher than 0.6. 
2. The price has to be less than  $20. 

Use the above conditions to filter the data for questions **2 and 3** below. 

**Questions:**
1. Where is Stellenbosch, anyway? How many wines from Stellenbosch are there in the *entire dataset*? 
2. *After filtering with the 2 conditions*, what is the average price of wine from the Bordeaux region? 
3. *After filtering with the 2 conditions*, what is the least expensive wine that's of the highest quality from the Okanagan Valley?



**Stretch Question:**
1. What is the average price of wine from Stellenbosch, according to the entire unfiltered dataset? 


**Note: Check the dataset to see if there are missing values; if there are, fill in missing values with the mean.**


In [43]:
# code here


### Challenge 2 (25 minutes)

**Challenge 21 from the 21DDC (Adapted)**

Dot wants to play retro video games with all their new friends! Help them figure out which games would be best.

Questions: 
    
1. What is the top 5 best selling games released before the year 2000.

     -  **Note**: Use Global_Sales
    
    
2. Create a new column called Aggregate_Score, which returns the proportional average between Critic Score and User_Score based on Critic_Count and User_Count. Plot a horizontal bar chart of the top 5 highest rated games by Aggregate_Score, not published by Nintendo before the year 2000. From this bar chart, what is the highest rated game by Aggregate_Score?

    -  **Note**: Critic_Count should be filled with the mean. User_Count should be filled with the median.
    
    
#### In the exercise above, there is some missing values in the dataset. Look up the pandas documentation to figure out how to fill missing values in a column. You will be using the **fillna()** function.   

In [44]:
# code here

# HINT

**How to create the Aggregate Score Column?**

\begin{equation*}
AggregateScore = \frac{(CriticCount * CriticScore)+(UserCount * UserScore)}{UserCount + CriticCount}
\end{equation*}

**Check Your Column Values**

The Critic_Score column is scored out of 100. The User_Score column is scored out of 10. You will need to modify one of the columns to match the other.

## Documentation

In the meantime, check out pandas the user guide in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide).

-------
**Why should I use the documentation?**

On the job as a data scientist or data analyst, more often than not, you may find yourself looking up the documentation of a particular function or plugin you use. Don't worry if there are a few functions you don't know by heart. However, there are just too many to know! An essential skill is to learn how to navigate documentation and understand how to apply the examples to your work. 

--------

Additional resources:

- To learn more about these topics, as well as other topics not covered here (e.g. reshaping, merging, additional subsetting methods, working with text data, etc.) check out [these introductory tutorials](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) from the `pandas` documentation
- To learn more about subsetting your data, check out [this tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset)
- This [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) may also be helpful as a reference.